In [3]:
import pandas as pd
import ormsgpack
import orjson
import re
import os

In [4]:
page_rank_results = pd.read_csv("..\\Page Rank Results\page_rank_results_with_urls.csv")
domain_rank_results = pd.read_csv("..\\Page Rank Results\domain_rank_results_with_domain_nm.csv")
citation_rank = pd.read_csv("..\\Page Rank Results\citation_ranks_with_scores.csv")
rps_info = pd.read_csv("..\\Data\Cord 19\metadata_cleaned.csv")

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:4: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:4: SyntaxWarning: invalid escape sequence '\C'
C:\Users\huzai\AppData\Local\Temp\ipykernel_20432\1504380687.py:1: SyntaxWarning: invalid escape sequence '\p'
  page_rank_results = pd.read_csv("..\\Page Rank Results\page_rank_results_with_urls.csv")
C:\Users\huzai\AppData\Local\Temp\ipykernel_20432\1504380687.py:2: SyntaxWarning: invalid escape sequence '\d'
  domain_rank_results = pd.read_csv("..\\Page Rank Results\domain_rank_results_with_domain_nm.csv")
C:\Users\huzai\AppData\Local\Temp\ipykernel_20432\1504380687.py:3: SyntaxWarning: invalid escape sequence '\c'
  citation_rank = pd.read_csv("..\\Page Rank Results\citation_ranks_with_sco

In [5]:
doc_id_to_url = {}
with open("..\\Data\\ind_to_url.json", "r" ) as f:
    doc_id_to_url = orjson.loads(f.read())

In [6]:
citation_rank.head()

,paper_title,citation_rank,Score
0,viral glycoproteins biological role and applic...,0.005979,10
1,bibliography of the current world literature,0.003455,10
2,author index volumes,0.001806,10
3,spezielle arzneimitteltherapie in der schwange...,0.001091,10
4,cumulative contents for,0.001031,10


In [7]:
citation_dict = dict(zip(citation_rank["paper_title"], citation_rank["Score"]))
rps_info_dict = dict(zip(rps_info["id"], zip(rps_info["title"], rps_info["url"])))
page_rank_dict = dict(zip(page_rank_results["URL"], page_rank_results["Score"]))
domain_rank_dict = dict(zip(domain_rank_results["Domain"], domain_rank_results["Score"]))
real_page_rank = dict(zip(page_rank_results["URL"], page_rank_results["Page Rank"]))
real_domain_rank = dict(zip(domain_rank_results["Domain"], domain_rank_results["Domain Score"]))
real_citation_rank = dict(zip(citation_rank["paper_title"], citation_rank["citation_rank"]))

In [8]:
with open("..\\Barrels\\barrels_index.json", "r", encoding="utf-8") as f:
    barrels_index = orjson.loads(f.read())

In [9]:
def word_lookup(indices):
    barrel_id = indices[0]
    word_index = indices[1]
    with open(f"..\\Barrels\\{barrel_id}.msgpack", "rb") as f:
        barrel_data = ormsgpack.unpackb(f.read())
    return barrel_data[word_index]

In [10]:
def normalize_title(title):
    title = title.lower()
    title = re.sub(r'\(.*?\)|\[.*?\]|\{.*?\}|<.*?>', ' ', title)
    title = re.sub(r'[^a-z\s]', ' ', title)
    title = re.sub(r'\s+', ' ', title)
    return title.strip()



In [11]:
def process_query(word, rps=True):
    text = re.sub(r'\n', ' ',word)
    if rps:
        text = re.sub(r'(?<!\d)[^\w\s]|[^\w\s](?!\d)', '', text)
    else:
        text = re.sub(r'(?<!\d)[^\w\s]|[^\w\s](?!\d)', ' ', text)
    text = re.sub(r"\s+", " ", text).strip()
    # Removing commas and circular brackets
    text = re.sub(r"[,\(\)\[\]\{\}]", "", text)
    text = text.lower()
    tokens = text.split(' ')
    return tokens

In [12]:
import math
from urllib.parse import urlparse
def score_html_files(hitlist):
    doc_id = hitlist[0]
    positions = hitlist[1]
    hit_counter = hitlist[2]

    n_title   = hit_counter[0]
    n_meta    = hit_counter[1]
    n_heading = hit_counter[2]
    n_total   = hit_counter[3]
    n_href    = hit_counter[4] # Anchor text
    in_domain = hit_counter[5] # Boolean (1 or 0)
    in_url    = hit_counter[6] # Boolean (1 or 0)
    doc_len   = hit_counter[7] if hit_counter[7] > 0 else 1 # Avoid div by zero

    # Initialize Score
    score = 0.0

    # ---------------------------------------------------------
    # PART 1: Zone Weighting (High value for Key Places)
    # Strategy: Use weighted sums, but cap them so one zone doesn't dominate.
    # ---------------------------------------------------------
    
    # Title: Huge weight. 1 match is great, 2 is max needed.
    score += min(n_title * 7.5, 15)
    
    # URL/Domain: If it's in the domain/URL, it's likely a main topic.
    if in_domain: score += 10
    if in_url:    score += 5
    
    # Headings: Important, but less than title.
    # Logic: 3 points per hit, capped at 9.
    score += min(n_heading * 3, 9)
    
    # Meta Description: Good signal for relevance.
    # Logic: 2 points per hit, capped at 6.
    score += min(n_meta * 2, 6)


    if positions:
        first_pos = positions[0]
        
        # if first_pos < 10:
        #     score += 15  # Immediate hit (e.g. first word)
        # elif first_pos < 50:
        #     score += 10  # Very early
        # elif first_pos < 100:
        #     score += 5   # Early
        # # If it appears after word 100, no position bonus.
        score += 15 - min(first_pos // 7, 15)

    # ---------------------------------------------------------
    # PART 3: Frequency & Spam Penalization
    # Strategy: Use Logarithmic Saturation on body text.
    # ---------------------------------------------------------
    
    # Calculate "Body Hits" (Total minus special zones to avoid double counting)
    # We want to score the remaining hits that weren't in title/headings
    body_hits = max(0, n_total - (n_title + n_heading + n_meta))
    
    # SPAM CHECK: Density
    # If the word makes up > 10% of the document, it's likely keyword stuffing.
    density = n_total / doc_len
    
   
    # AGGRESSIVE SCORING: 
    # 1 hit   = 0 pts  -> 4.8 pts (Immediate boost)
    # 5 hits  = 7 pts  -> 12.5 pts (Good relevance)
    # 15 hits = 11 pts -> 19.4 pts (Max relevance)
    freq_score = math.log(1 + body_hits) * 7        
    score += min(freq_score, 20)

    score *= (1 - density
)
    # ---------------------------------------------------------
    # PART 4: Final Clamping (Scale 1-80)
    # ---------------------------------------------------------
    
    # Ensure minimum score of 1
    final_score = max(1.0, score)
    
    # Hard cap at 80
    final_score = min(80.0, final_score)

    doc_url = doc_id_to_url.get(doc_id.replace("H",""), "")
    page_rank_score = page_rank_dict.get(doc_url, 0)
    domain = urlparse(doc_url).netloc
    domain_rank_score = domain_rank_dict.get(domain, 0)

    return int(final_score + page_rank_score + domain_rank_score)

In [13]:
def rank_research_papers(hitlist):
    doc_id = hitlist[0]
    positions = hitlist[1]
    hit_counter = hitlist[2]
    # Mapping hit_counter indices
    n_golden = hit_counter[0]  # Title, Author, Abstract (The "Golden" Zone)
    n_body   = hit_counter[1]  # Body Text
    n_other  = hit_counter[2]  # References, acknowledgments
    n_total  = hit_counter[3]
    doc_len  = hit_counter[4]

    # Initialize Score
    score = 0.0

    # ---------------------------------------------------------
    # PART 1: The Golden Zone (Title, Author, Abstract)
    # Strategy: This is the strongest signal. High multiplier.
    # ---------------------------------------------------------
    
    # Logic: 5 points per hit in title and abstract 
    # We cap this at 35
    score += min(n_golden * 5, 35)

    # ---------------------------------------------------------
    # PART 2: Position Boosting ("The Hook")
    # Strategy: Papers define their topic in the first paragraph.
    # ---------------------------------------------------------
    if positions:
        first_pos = positions[0]
        
        # If the word appears in the first 50 words, it's part of the intro/abstract.
        # if first_pos < 50:
        #     score += 10
        # elif first_pos < 150:
        #     score += 5
        score += 15 - min(first_pos // 15, 10)
    
    # ---------------------------------------------------------
    # PART 3: Body Frequency (The Content)
    # Strategy: Aggressive Logarithmic Saturation.
    # ---------------------------------------------------------
    
    # SPAM/DENSITY CHECK: 
    # Papers are dense, but if a word is > 20% of the text, it's likely a parsing error or bad data.
    density = n_total / doc_len
    
    
    relevant_hits = n_body + (n_other * 0.1) 
    
    freq_score = math.log(1 + relevant_hits) * 10
    
    
    score += min(freq_score, 40)

    score *= (1-density)

    
    final_score = max(1.0, score)
    final_score = min(80.0, final_score)
    title = rps_info_dict.get(int(doc_id.replace("P","")), ("", ""))[0].strip()
    title = normalize_title(title)
    citation_rank = citation_dict.get(title, 0)
    return int(final_score + citation_rank)

In [14]:
def perform_single_word_search(word):
    tokens = process_query(word, rps=False)
    if not tokens:
        return []
    token = tokens[0]
    
    if token not in barrels_index:
        return []
    
    indices = barrels_index[token]
    hitlist = word_lookup(indices)
    
    html_hits = []
    rps_hits = []
    
    for hit in hitlist:
        doc_id = hit[0]
        if doc_id.startswith("H"):
            html_hits.append(hit)
        elif doc_id.startswith("P"):
            rps_hits.append(hit)
    
    # Score HTML hits
    scored_html = []
    for hit in html_hits:
        score = score_html_files(hit)
        scored_html.append((hit[0], score, doc_id_to_url.get(hit[0].replace("H",""), "")))
    
    # Score RPS hits
    scored_rps = []
    for hit in rps_hits:
        score = rank_research_papers(hit)
        scored_rps.append((hit[0], score, rps_info_dict.get(int(hit[0].replace("P","")), ("", ""))[1]))
    
    # Combine and sort results
    combined_results = scored_html + scored_rps
    combined_results.sort(key=lambda x: x[1], reverse=True)
    
    return combined_results

In [15]:
perform_single_word_search("9/11")

[('H45973', 65, 'https://en.wikipedia.org/wiki/9/11_(2002_film)'),
 ('H44810',
  53,
  'https://www.huffpost.com/entry/post-911-islam-converts_n_935572'),
 ('H45937',
  48,
  'https://en.wikipedia.org/wiki/Saddam_Hussein_and_al-Qaeda_link_allegations'),
 ('H45834',
  39,
  'https://en.wikipedia.org/wiki/World_Trade_Center_(1973%E2%80%932001)'),
 ('H44952', 37, 'https://en.wikiquote.org/wiki/Fred_Phelps'),
 ('H46270',
  37,
  'https://en.wikipedia.org/wiki/Iran_and_state-sponsored_terrorism'),
 ('H40228', 36, 'https://en.wikipedia.org/wiki/Islamic_terrorism'),
 ('H36350', 35, 'https://en.wikipedia.org/wiki/Madrassas_in_Pakistan'),
 ('H35620', 34, 'https://en.wikipedia.org/wiki/Donald_J._Trump_Foundation'),
 ('H36533', 33, 'https://en.wikipedia.org/wiki/Persecution_of_Muslims'),
 ('H37104', 33, 'https://en.wikipedia.org/wiki/Pax_Americana'),
 ('H41527', 33, 'https://en.wikipedia.org/wiki/Framing_(social_sciences)'),
 ('H38737',
  32,
  'https://en.wikipedia.org/wiki/Islamic%E2%80%93Jewis

In [16]:
perform_single_word_search("health")

[('H27607',
  92,
  'https://health.ec.europa.eu/non-communicable-diseases/mental-health_en'),
 ('H12634', 91, 'https://www.health.harvard.edu/special-health-reports'),
 ('H13295',
  90,
  'https://www.health.harvard.edu/mens-health/a-guide-to-mens-health-fifty-and-forward'),
 ('H15462',
  90,
  'https://www.health.harvard.edu/womens-health/a-guide-to-womens-health-fifty-and-forward'),
 ('H17054',
  90,
  'https://www.health.harvard.edu/staying-healthy/dental-health-for-adults'),
 ('H31156',
  90,
  'https://health.ec.europa.eu/state-health-eu/health-glance-europe/health-glance-europe-2018_en'),
 ('P10192', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6534252/'),
 ('P10267', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6568391/'),
 ('P10303', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6591856/'),
 ('P10638', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6724249/'),
 ('P10713', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6766041/'),
 ('P10815', 90, 'https://ww

In [17]:
perform_single_word_search("cancer")

[('P10450', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6640725/'),
 ('P10878', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6820718/'),
 ('P10988', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6865829/'),
 ('P11159', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6942430/'),
 ('P11307', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7005297/'),
 ('P1223', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2923618/'),
 ('P12452', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7087863/'),
 ('P13266', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7091791/'),
 ('P13727', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7101997/'),
 ('P1380', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3024302/'),
 ('P14336', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7109844/'),
 ('P15049', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7140054/'),
 ('P15933', 90, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7165995/'),
 ('P17879', 90, 'https://www.ncbi.nlm.ni

In [22]:
def multi_word_queries(query):
    tokens_1 = process_query(query, rps=True)
    hitlists = []
    for token in tokens_1:
        if token in barrels_index:
            barrel_indices = barrels_index[token]
            htl = word_lookup(barrel_indices)
            hitlists.append(htl)
    return hitlists

In [24]:
def get_intersected_results(hitlists):
    """
    Takes a list of hitlists (one per word) and returns ONLY documents 
    that appear in ALL hitlists.
    """
    
    # 1. Safety Checks
    if not hitlists:
        return {}
    
    # 2. Optimization: Sort the hitlists by length!
    # We want to process the shortest list first to reduce the search space immediately.
    # hitlists is a list of lists. We sort based on len(list).
    hitlists.sort(key=len)
    
    # 3. Create the initial set of candidate Doc IDs from the shortest list
    # hit[0] is the doc_id based on your structure
    common_doc_ids = { hit[0] for hit in hitlists[0] }
    
    # 4. Iteratively intersect with the remaining lists
    for i in range(1, len(hitlists)):
        # Extract IDs from the next list
        next_doc_ids = { hit[0] for hit in hitlists[i] }
        
        # Perform Intersection (AND operation)
        # This updates common_doc_ids to keep ONLY those present in both
        common_doc_ids.intersection_update(next_doc_ids)
        
        # Optimization: If we hit 0 results early, stop!
        if not common_doc_ids:
            return {}

    # ---------------------------------------------------------
    # PART 5: Data Reconstruction
    # Now we have the list of DocIDs that contain ALL words.
    # But we lost the position/counter data! We need to grab it back 
    # so we can rank them properly.
    # ---------------------------------------------------------
    
    final_data = {}
    
    # Iterate through ALL original hitlists
    for hitlist in hitlists:
        for hit in hitlist:
            doc_id = hit[0]
            
            # Only process if this doc survived the intersection
            if doc_id in common_doc_ids:
                if doc_id not in final_data:
                    final_data[doc_id] = []
                
                # Append the hit data (positions, counters) for this word
                final_data[doc_id].append(hit)
                
    # Returns: { "H1": [ [hit_data_word1], [hit_data_word2] ], ... }
    return final_data

In [34]:
import heapq

def closest_cluster(lists):
    """
    Finds the cluster containing one number from each list such that
    the maximum difference among numbers is minimized.
    """
    k = len(lists)
    pointers = [0] * k  # current index for each list
    # Make a min-heap: (value, list_index, element_index)
    heap = []
    current_max = float('-inf')

    # Initialize heap with first element of each list
    for i, lst in enumerate(lists):
        val = lst[0]
        heapq.heappush(heap, (val, i, 0))
        current_max = max(current_max, val)

    best_range = float('inf')
    best_cluster = []

    while True:
        current_min, list_idx, elem_idx = heapq.heappop(heap)
        # Check current range
        if current_max - current_min < best_range:
            best_range = current_max - current_min
            # Collect current elements
            best_cluster = [heap[i][0] for i in range(len(heap))] + [current_min]
        
        # Move pointer in the list that contributed the min
        if elem_idx + 1 < len(lists[list_idx]):
            next_val = lists[list_idx][elem_idx + 1]
            heapq.heappush(heap, (next_val, list_idx, elem_idx + 1))
            current_max = max(current_max, next_val)
        else:
            # One list exhausted → done
            break

    return best_cluster, best_range



In [138]:
htls  = multi_word_queries("machine learning")

In [139]:
htls_intersected = get_intersected_results(htls)

In [140]:
grouped_htls = {}

In [142]:
rslts = []

for htls in htls_intersected:
    # Extract positions and flatten if they are lists
    position_vectors = []
    for i in range(len(htls_intersected[htls])):
        val = htls_intersected[htls][i][1]
        if isinstance(val, list):
            position_vectors.extend(val)
        else:
            position_vectors.append(val)

    # Keep order as in the document
    best_cluster = position_vectors
    cluster_range = max(best_cluster) - min(best_cluster) if best_cluster else 0

    # Count consecutive "very close" matches (diff <= 2)
    # Only count if words appear in proximity (consecutively/near each other)
    close_count = 0
    if len(best_cluster) > 1:
        sorted_positions = sorted(best_cluster)
        for i in range(len(sorted_positions) - 1):
            # Check if consecutive words are within 2 positions of each other
            if sorted_positions[i+1] - sorted_positions[i] <= 2:
                close_count += 1

    # Fetch URL
    if htls.startswith("P"):
        url = rps_info_dict.get(int(htls.replace("P", "")), ("", ""))[1]
    else:
        url = doc_id_to_url.get(htls.replace("H", ""), "")

    # Print and append
    print(f"{htls}:  range: {cluster_range} - close_matches: {close_count} - {url}")
    rslts.append((htls, cluster_range, close_count, url))


P10084:  range: 3717 - close_matches: 1 - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6494680/
P10090:  range: 1 - close_matches: 1 - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6499107/
P10119:  range: 8848 - close_matches: 9 - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6515310/
P10120:  range: 1 - close_matches: 1 - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6516830/
P10169:  range: 6830 - close_matches: 8 - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6528302/
P10235:  range: 6177 - close_matches: 13 - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6556904/
P1025:  range: 5483 - close_matches: 2 - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2836373/
P10258:  range: 5454 - close_matches: 2 - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6563321/
P1027:  range: 1151 - close_matches: 0 - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2837245/
P10355:  range: 1 - close_matches: 1 - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6613463/
P10411:  range: 7444 - close_matches: 15 - https:/

In [143]:
# Sort by score
rslts.sort(key=lambda x: x[2], reverse=True)

In [144]:
rslts

[('H12360', 163, 18, 'https://futurism.com/category/machine-learning/page/2'),
 ('H11040', 236, 17, 'https://futurism.com/category/machine-learning'),
 ('H6744',
  1048,
  17,
  'https://neurosys.com/blog/machine-learning-project-with-ai-company'),
 ('H13901',
  488,
  16,
  'https://time.com/collections/the-ai-dictionary-from-allbusiness-com/7273968/definition-of-machine-learning/'),
 ('H16803', 935, 16, 'https://futurism.com/ai-investor-agi-warning'),
 ('P10411', 7444, 15, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6631137/'),
 ('P28341', 4293, 15, 'https://doi.org/10.1101/2020.02.03.932350'),
 ('P29429', 1883, 15, 'https://doi.org/10.1101/2020.04.03.024885'),
 ('P29470', 1950, 15, 'https://doi.org/10.1101/2020.04.04.20052092'),
 ('P30919', 1825, 15, 'https://doi.org/10.1101/2020.05.02.20088427'),
 ('P416', 8353, 15, 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2244602/'),
 ('H11904',
  599,
  15,
  'https://futurism.com/the-byte/pandemic-confused-machine-learning-systems'),
 ('H

In [183]:
def rank_multiword_results(query):
    """
    Simple and efficient multi-word query ranking:
    1. Parse query and intersect hitlists
    2. Calculate word-level scores (using existing functions)
    3. Average the scores
    4. Count close matches (words within 2 positions)
    5. Calculate combined score: close_matches + avg_word_score
    6. Rank documents by combined score
    """
    
    # =====================================================================
    # STEP 1: Parse query and get hitlists
    # =====================================================================
    tokens = process_query(query, rps=True)
    hitlists = []
    
    for token in tokens:
        if token in barrels_index:
            barrel_indices = barrels_index[token]
            htl = word_lookup(barrel_indices)
            hitlists.append(htl)
    
    if not hitlists:
        return []
    
    # =====================================================================
    # STEP 2: Perform barrel intersection
    # =====================================================================
    hitlists_sorted = sorted(hitlists, key=len)
    common_doc_ids = {hit[0] for hit in hitlists_sorted[0]}
    
    for i in range(1, len(hitlists_sorted)):
        next_doc_ids = {hit[0] for hit in hitlists_sorted[i]}
        common_doc_ids.intersection_update(next_doc_ids)
        if not common_doc_ids:
            return []
    
    # Reconstruct hit data
    intersected_data = {}
    for hitlist in hitlists:
        for hit in hitlist:
            doc_id = hit[0]
            if doc_id in common_doc_ids:
                if doc_id not in intersected_data:
                    intersected_data[doc_id] = []
                intersected_data[doc_id].append(hit)
    
    # =====================================================================
    # STEP 3 & 4: Calculate word scores and close matches
    # =====================================================================
    ranked_results = []
    
    for doc_id in intersected_data:
        hits_for_doc = intersected_data[doc_id]
        
        # Calculate individual word scores
        word_scores = []
        position_vectors = []
        
        for hit in hits_for_doc:
            # Score using existing functions
            if doc_id.startswith("P"):
                word_score = rank_research_papers(hit)
            else:
                word_score = score_html_files(hit)
            
            word_scores.append(word_score)
            
            # Collect positions
            positions = hit[1]
            if isinstance(positions, list):
                position_vectors.extend(positions)
            else:
                position_vectors.append(positions)
        
        # Average word score
        avg_word_score = sum(word_scores) / len(word_scores) if word_scores else 0
        
        # Count close matches (words within 2 positions)
        close_count = 0
        
        if len(position_vectors) > 1:
            sorted_positions = sorted(position_vectors)
            for i in range(len(sorted_positions) - 1):
                if sorted_positions[i+1] - sorted_positions[i] <= 2:
                    close_count += 1
        
        # Calculate cluster range
        cluster_range = max(position_vectors) - min(position_vectors) if position_vectors else 0
        
        # =====================================================================
        # STEP 5: Calculate combined score
        # Score includes:
        # - close_count: number of word pairs within 2 positions
        # - avg_word_score: relevance of individual words
        # =====================================================================
        combined_score = close_count + avg_word_score
        
        # Fetch URL
        if doc_id.startswith("P"):
            url = rps_info_dict.get(int(doc_id.replace("P", "")), ("", ""))[1]
        else:
            url = doc_id_to_url.get(doc_id.replace("H", ""), "")
        
        ranked_results.append({
            'doc_id': doc_id,
            'combined_score': combined_score,
            'avg_word_score': avg_word_score,
            'close_matches': close_count,
            'cluster_range': cluster_range,
            'url': url,
            'positions': position_vectors
        })
    
    # =====================================================================
    # STEP 6: Sort by combined score
    # =====================================================================
    ranked_results.sort(key=lambda x: x['combined_score'], reverse=True)
    
    return ranked_results


In [184]:
# Test the simplified rank_multiword_results function
mw_results = rank_multiword_results("pakistan india war 2025")

# Display results in a formatted way
print(f"\n{'='*130}")
print(f"Multi-Word Query Results: 'pakistan india war 2025'")
print(f"{'='*130}")
print(f"{'Doc ID':<10} {'Combined':<12} {'Close':<8} {'Avg Score':<12} {'Range':<8} {'URL':<60}")
print(f"{'-'*130}")

for r in mw_results[:15]:  # Show top 15
    doc_id = r['doc_id']
    combined = r['combined_score']
    close = r['close_matches']
    score = r['avg_word_score']
    range_ = r['cluster_range']
    url = r['url'][:57] + "..." if len(r['url']) > 60 else r['url']
    
    print(f"{doc_id:<10} {combined:<12.1f} {close:<8} {score:<12.1f} {range_:<8} {url:<60}")



Multi-Word Query Results: 'pakistan india war 2025'
Doc ID     Combined     Close    Avg Score    Range    URL                                                         
----------------------------------------------------------------------------------------------------------------------------------
H41974     79.8         18       61.8         13592    https://en.wikipedia.org/wiki/2025_India%E2%80%93Pakistan...
H34337     64.2         12       52.2         23064    https://en.wikipedia.org/wiki/Pakistan_Army                 
H46187     63.2         8        55.2         1639     https://www.aljazeera.com/news/2025/5/10/india-and-pakist...
H18528     51.2         3        48.2         1321     https://www.geo.tv/latest/628405-implications-of-afghan-p...
H34300     51.2         5        46.2         6781     https://en.wikipedia.org/wiki/Portal:Pakistan               
H40241     51.0         8        43.0         23087    https://en.wikipedia.org/wiki/Second_Nagorno-Karabakh_War   
H461

In [187]:
# Test with a different query
mw_results_2 = rank_multiword_results("Geo news")

print(f"\n{'='*130}")
print(f"Multi-Word Query Results: 'Geo news'")
print(f"{'='*130}")
print(f"{'Doc ID':<10} {'Combined':<12} {'Close':<8} {'Avg Score':<12} {'Range':<8} {'URL':<60}")
print(f"{'-'*130}")

for r in mw_results_2[:15]:  # Show top 15
    doc_id = r['doc_id']
    combined = r['combined_score']
    close = r['close_matches']
    score = r['avg_word_score']
    range_ = r['cluster_range']
    url = r['url'][:57] + "..." if len(r['url']) > 60 else r['url']
    
    print(f"{doc_id:<10} {combined:<12.1f} {close:<8} {score:<12.1f} {range_:<8} {url:<60}")



Multi-Word Query Results: 'Geo news'
Doc ID     Combined     Close    Avg Score    Range    URL                                                         
----------------------------------------------------------------------------------------------------------------------------------
H27761     90.5         8        82.5         461      https://hamariweb.com/pakistan-tv-channels/geo_news.aspx    
H10992     84.0         8        76.0         1056     https://apps.apple.com/pk/app/geo-news-official/id617329127 
H10402     77.5         14       63.5         216      https://www.geo.tv/shows/geo-headlines/633696-geo-news-12...
H10029     73.5         7        66.5         940      https://www.geo.tv                                          
H10156     72.5         14       58.5         246      https://www.geo.tv/tv-videos                                
H10276     69.0         15       54.0         668      https://www.geo.tv/privacy-policy-and-tos                   
H10330     68.0    

In [188]:
# Test with a different query
mw_results_3 = rank_multiword_results("machine learning algorithms")

print(f"\n{'='*130}")
print(f"Multi-Word Query Results: 'machine learning algorithms'")
print(f"{'='*130}")
print(f"{'Doc ID':<10} {'Combined':<12} {'Close':<8} {'Avg Score':<12} {'Range':<8} {'URL':<60}")
print(f"{'-'*130}")

for r in mw_results_3[:15]:  # Show top 15
    doc_id = r['doc_id']
    combined = r['combined_score']
    close = r['close_matches']
    score = r['avg_word_score']
    range_ = r['cluster_range']
    url = r['url'][:57] + "..." if len(r['url']) > 60 else r['url']
    
    print(f"{doc_id:<10} {combined:<12.1f} {close:<8} {score:<12.1f} {range_:<8} {url:<60}")



Multi-Word Query Results: 'machine learning algorithms'
Doc ID     Combined     Close    Avg Score    Range    URL                                                         
----------------------------------------------------------------------------------------------------------------------------------
P29470     88.0         23       65.0         1950     https://doi.org/10.1101/2020.04.04.20052092                 
P30919     79.3         15       64.3         7115     https://doi.org/10.1101/2020.05.02.20088427                 
H13901     78.7         16       62.7         696      https://time.com/collections/the-ai-dictionary-from-allbu...
H6744      78.7         18       60.7         2069     https://neurosys.com/blog/machine-learning-project-with-a...
H26051     76.7         9        67.7         3192     https://machinelearning.apple.com/research/iclr-2025        
H18068     76.7         13       63.7         1603     https://machinelearning.apple.com/highlights                
